In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt
from string import punctuation
import os
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from pandas import DataFrame

/Users/hp/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hp/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
cwd = os.getcwd()
datadir = '/'.join(cwd.split('/')) + '/data/'
data = pd.read_csv(datadir + 'labeled_data.csv', sep=',',index_col=0)
tweets=data

In [11]:
data.head(100)

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [26]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
tweets=data['tweet']

for tweet in tweets[:20]:
    tokens = tweet.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    ps = PorterStemmer()
    for i in range(len(tokens)):
        tokens[i] = ps.stem(tokens[i])
    #print(tokens)
    #print('.........................')

['RT', 'mayasolovely', 'As', 'woman', 'shouldnt', 'complain', 'cleaning', 'house', 'amp', 'man', 'always', 'take', 'trash']
.........................
['RT', 'boy', 'dats', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place']
.........................
['RT', 'UrKindOfBrand', 'Dawg', 'RT', 'You', 'ever', 'fuck', 'bitch', 'start', 'cry', 'You', 'confused', 'shit']
.........................
['RT', 'CGAnderson', 'vivabased', 'look', 'like', 'tranny']
.........................
['RT', 'ShenikaRoberts', 'The', 'shit', 'hear', 'might', 'true', 'might', 'faker', 'bitch', 'told', 'ya']
.........................
['TMadisonx', 'The', 'shit', 'blows', 'meclaim', 'faithful', 'somebody', 'still', 'fucking', 'hoes']
.........................
['BrighterDays', 'sit', 'HATE', 'another', 'bitch', 'got', 'much', 'shit', 'going']
.........................
['cause', 'Im', 'tired', 'big', 'bitches', 'coming', 'us', 'skinny']
.........................
['amp', 'might', 'get', 'ya', 'bitch', 'back', 'amp', 

['got', 'number', 'Im', 'bitch', 'boy']
.........................
['know', 'shit', 'eatin', 'grin', 'You', 'either', 'fucked', 'girl', 'dreams', 'sold', 'car', 'We', 'know', 'youre', 'faggot', 'Whatd']
.........................
['older', 'men', 'cause', 'theyre', 'mature', 'That', 'supposed', 'fool', 'us', 'bitch']
.........................
['prolly', 'leave', 'bitch', 'cups', 'hand', 'yo', 'hand', 'hand']
.........................
['said', 'dont', 'like', 'tastes', 'like', 'wolf', 'pussy', 'How', 'even', 'know', 'tastes', 'like']
.........................
['taught', 'eat', 'pussy', 'cut', 'hair', 'taught', 'eat', 'pussy', 'Name', 'movie']
.........................
['used', 'dress', 'like', 'cop', 'Oh', 'really', 'Yeahright', 'pussy', 'Really', 'YesIll', 'NEVER', 'FORGET']
.........................
['went', 'jail', 'flock', 'amp', 'came', 'home', 'wit', 'strike', 'You', 'went', 'jail', 'bitch', 'came', 'home', 'dike', 'YG']
.........................
['fuck', 'stand', 'dizzy', 'bitch', 

['bruh', 'hoes', 'gotta', 'stupid']
.........................
['SOMEBODY', 'GONNA', 'DIE', 'RT', 'theres', 'bitch', 'NJT', 'setting', 'lean']
.........................
['Yall', 'boring', 'Nope', 'lick', 'side', 'bitch', 'face', 'Im', 'hittin']
.........................
['Of', 'course', 'cooders', 'nudes', 'trash', 'She', 'lives', 'DMV', 'expect']
.........................
['My', 'stomach', 'pain', 'like', 'mufucka', 'RT', 'OwIPharaoh', 'This', 'nigga', 'thats', 'rating', 'bitches']
.........................
['bitches', 'get', 'stuff', 'done']
.........................
['fucc', 'nicca', 'yu', 'pose', 'pullin']
.........................
['maaaaannnn', 'hope', 'RT', 'VonshayeB', 'get', 'rid', 'John', 'Legend', 'looking', 'Charlie', 'Batch']
.........................
['mike', 'calls', 'tbird', 'mikemastroccco']
.........................
['niggas', 'sending', 'screenshots', 'bitches', 'like', 'stop', 'texting', 'cuh']
.........................
['white', 'bitch', 'think', 'miley', 'cyrus']
.

.........................
['joelxmontoya', 'Im', 'dreaming', 'must', 'fucked']
.........................
['SkinnyBee', 'LMAO', 'bitch', 'listening', 'gospel', 'music', 'liquor', 'hand', 'crying']
.........................
['amanda', 'bitch', 'ass', 'eva', 'cheat', 'fizz', 'fine', 'ass', 'ion', 'theres', 'nigga', 'looks', 'Thought', 'yall', 'knew']
.........................
['amp', 'feel', 'disrespected', 'bitch', 'try', 'suck', 'dry', 'lips']
.........................
['Yall', 'bitches', 'switch', 'roo', 'niggas', 'hell']
.........................
['Who', 'RT', 'Yall', 'hype', 'bread', 'loaf', 'chick', 'Im', 'dyke', 'rapper']
.........................
['Cowboys', 'win', 'games', 'regular', 'season', 'get', 'playoffs', 'trash', 'af', 'hating', 'ass', 'nigga']
.........................
['Having', 'bitches', 'really', 'got', 'hoes']
.........................
['Niggas', 'want', 'good', 'girl', 'still', 'wanna', 'fuck', 'hoes', 'aint', 'gone', 'work', 'Gotta', 'pick', 'boy', 'stfu']
.......

['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'Brooklyn', 'Park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'Brooklyn', 'Park', 'slope', 'TOP', 'RATED', 'SPA', 'httptcoyRfdaIooYZ']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........................
['BEST', 'ASIAN', 'MASSAGE', 'ON', 'THE', 'park', 'slope', 'TOP', 'RATED', 'SPA']
.........

...

.........................
['Wait', 'min', 'nigga', 'bitch', 'Lol', 'RT', 'FuckTheSteelers', 'HTTR', 'dont', 'give', 'damn', 'cont']
.........................
['Wait', 'minute', 'Bowyer', 'causing', 'yellow', 'Chase', 'race', 'RT', 'Caution', 'NASCAR']
.........................
['Wait', 'bitchKeep', 'simple', 'never', 'basic']
.........................
['Wait', 'kidfivevibes', 'take', 'brownies']
.........................
['Wait', 'thats', 'noooo', 'butt', 'naked', 'bitch', 'awwwww', 'boy']
.........................
['Wait', 'What', 'mudda', 'cunt', 'hypocrisy']
.........................
['Wait', 'What', 'RT', 'KingHorseDick', 'ever', 'put', 'ur', 'finger', 'lil', 'hole', 'back', 'pussy', 'Cool']
.........................
['Waiting', 'brownies', 'want', 'brownies', 'Someone', 'hooks']
.........................
['Waiting', 'bitches', 'badnewsbridget', 'MorbidMermaid']
.........................
['Waiting', 'wetback', 'sycksyllables']
.........................
['Waka', 'bitch', 'bad', 'fr'

['Women', 'want', 'Best', 'Man', 'Holiday', 'type', 'nigga', 'meanwhile', 'mad', 'THOT', 'pics', 'ya', 'gram', 'talking', 'could', 'us', 'bitch', 'die']
.........................
['Women', 'feminist', 'ugly', 'bitches', 'cant', 'find', 'man']
.........................
['Wont', 'catch', 'sneak', 'dissing', 'thats', 'fo', 'birds']
.........................
['Wont', 'step', 'dad', 'hoe', 'Lol']
.........................
['Woot', 'RT', 'Yankees', 'Yankees', 'completed', 'sixth', 'series', 'sweep', 'season', 'fourth', 'home']
.........................
['Word', 'RT', 'MaxinBetha', 'And', 'pussy', 'RT', 'HumbltonBanks', 'And', 'snacks', 'RT', 'AerinMarie', 'Ladies', 'keep', 'beer', 'cont']
.........................
['Word', 'Ray', 'Rice', 'RT', 'marry', 'aint', 'gone', 'divorces', 'bitch', 'way', 'outta', 'box']
.........................
['Word', 'bird']
.........................
['Word', 'doe', 'GM', 'RT', 'Good', 'morningits', 'celebration', 'bitches']
.........................
['Word', 'Bu

['Your', 'girls', 'pussy', 'smells', 'like', 'sour', 'cream', 'onions']
.........................
['Your', 'grammar', 'trash']
.........................
['Your', 'leader', 'rocks', 'fucking', 'mo', 'hawk', 'dances', 'like', 'faggot', 'ATL', 'HoustonRockets']
.........................
['Your', 'name', 'follows', 'word', 'cunt', 'dictionary', 'sooo', 'whats', 'tell']
.........................
['Your', 'new', 'bitch', 'old', 'hoe']
.........................
['Your', 'pathetic', 'plan', 'failed', 'Enjoy', 'phone', 'fucking', 'monkey', 'Let', 'see', 'ass', 'Benton', 'park', 'You', 'wanna', 'talk', 'shit', 'Ill', 'carve', 'face']
.........................
['Your', 'pussy', 'hella', 'tight', 'Took', 'hour', 'slide', 'mothafucka', 'Quote', 'Ill', 'compliment']
.........................
['Your', 'pussy', 'hole', 'must', 'small', 'adjustment', 'phase', 'RT', 'Tf', 'gentle', 'intercourse']
.........................
['Your', 'retarded']
.........................
['Your', 'vaginal', 'walls', 'infla

['dolled', 'got', 'voodoo', 'bitches']
.........................
['dont', 'buy', 'box', 'colored', 'goldfish', 'racist', 'tbh']
.........................
['eat', 'fried', 'eggs', 'leave', 'yolk', 'entirely', 'yellow', 'ur', 'disgrace', 'also', 'nasty']
.........................
['get', 'mad', 'ur', 'feelings', 'bout', 'Zoes', 'vs', 'Yanks', 'tweets', 'twitter', 'aint', 'rolling', 'shit']
.........................
['gon', 'wit', 'ur', 'bae', 'even', 'cheat', 'shut', 'fuck', 'take', 'dick', 'pussy', 'aint', 'nothing', 'better', 'love']
.........................
['uckin', 'love', 'freebees', 'like', 'pom', 'Im', 'duck', 'hm', 'life']
.........................
['ur', 'girl', 'watch', 'Snapped', 'front', 'trying', 'spook']
.........................
['yo', 'girl', 'doesnt', 'swallow', 'kids', 'bitch', 'basic']
.........................
['fuck', 'bitch', 'cause', 'nigga', 'im', 'shit', 'Ill', 'salute', 'ya']
.........................
['delete', 'tweets', 'one', 'hour', 'tweet', 'gigantic', 'p

.........................
['today', 'friend', 'taught', 'hood', 'slang', 'said', 'fuck', 'nigga', 'nigga', 'bitch', 'fuck', 'boy', 'nigga', 'bruh']
.........................
['tom', 'ford', 'probably', 'fucking', 'hoes']
.........................
['bad', 'ur', 'pussy']
.........................
['many', 'bitches', 'enough', 'adults']
.........................
['many', 'grown', 'man', 'worryin', 'friendzoneyall', 'got', 'priorities', 'fucked', 'nig']
.........................
['top', 'layer', 'trash']
.........................
['top', 'notch', 'hoes', 'get', 'lesser']
.........................
['trash', 'RT', 'AyeoFool', 'Easily', 'Real', 'Gs', 'move', 'silence', 'like', 'lasagna', 'line', 'RT', 'SHAWNHINDRIIX', 'RT', 'WIT', 'WAYNE', 'LINE', 'THAT', 'WENT', 'OVER', 'NIGGAS', 'HEADS']
.........................
['trash', 'RT', 'VibeMagazine', 'Can', 'Notorious', 'son', 'CJ', 'Wallace', 'rap', 'Watch', 'amp', 'vote']
.........................
['trash', 'af']
.........................
['tra

In [35]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
tweets=data['tweet']

for tweet in tweets[:20]:
    tokens = tweet.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    #stem the tokens
    ps = PorterStemmer()
    for i in range(len(tokens)):
        tokens[i] = ps.stem(tokens[i])
    #print(tokens)
    #print('.........................')
    data_x

['RT', 'mayasolov', 'As', 'woman', 'shouldnt', 'complain', 'clean', 'hous', 'amp', 'man', 'alway', 'take', 'trash']
.........................
['RT', 'boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'place']
.........................
['RT', 'urkindofbrand', 'dawg', 'RT', 'you', 'ever', 'fuck', 'bitch', 'start', 'cri', 'you', 'confus', 'shit']
.........................
['RT', 'cganderson', 'vivabas', 'look', 'like', 'tranni']
.........................
['RT', 'shenikarobert', 'the', 'shit', 'hear', 'might', 'true', 'might', 'faker', 'bitch', 'told', 'ya']
.........................
['tmadisonx', 'the', 'shit', 'blow', 'meclaim', 'faith', 'somebodi', 'still', 'fuck', 'hoe']
.........................
['brighterday', 'sit', 'hate', 'anoth', 'bitch', 'got', 'much', 'shit', 'go']
.........................
['caus', 'Im', 'tire', 'big', 'bitch', 'come', 'us', 'skinni']
.........................
['amp', 'might', 'get', 'ya', 'bitch', 'back', 'amp', 'that']
.........................


game
game
game
game


'black'

In [36]:
def prepare_data(train_data, mode):
	# create the tokenizer
	tokenizer = Tokenizer()
	# fit the tokenizer on the documents
	tokenizer.fit_on_texts(train_data)
	# encode training data set
	Xtrain = tokenizer.texts_to_matrix(train_data, mode=mode)
	return Xtrain

In [ ]:
prepare_data(train_data, mode)

In [ ]:
# prepare bag of words encoding of docs




# load all training reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, True)
negative_lines = process_docs('txt_sentoken/neg', vocab, True)
train_docs = positive_lines + negative_lines
# load all test reviews
positive_lines = process_docs('txt_sentoken/pos', vocab, False)
negative_lines = process_docs('txt_sentoken/neg', vocab, False)
test_docs = positive_lines + negative_lines
# prepare labels
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])

modes = ['binary', 'count', 'tfidf', 'freq']
results = DataFrame()
for mode in modes:
	# prepare data for mode
	Xtrain, Xtest = prepare_data(train_docs, test_docs, mode)
	# evaluate model on data for mode
	results[mode] = evaluate_mode(Xtrain, ytrain, Xtest, ytest)
# summarize results
print(results.describe())
# plot results
results.boxplot()
pyplot.show()